# Subterranea: Day-Knights and Night-Knights 
This notebook explores a set of logic puzzles inspired by some puzzle types found in Raymond Smullyan's "To Mock a Mockingbird".

> In the city of Subterranea the inhabitants are of two types: day-knights and night-knights. Day-knights tell the truth during the day and lie at night, night-knights tell the truth at night and lie during the day.

First we define some functions to create and manipulate the data structures. We represent the state of affairs as a triple like day-day-day where the first entry says whether it is day or night, the second entry says if the first person is a day-knight or a night-knight, and the third entry says whether the second person is a day-knight or a night-knight.

In [43]:
import math

time = ['day', 'night']
options = []

for i in time:
    for j in time:
        for k in time:
            options.append([i,j,k])

def truthValue(person, option):
    return option[0] == option[person]

def firstInhabitant(option):
    return truthValue(1,option)

def secondInhabitant(option):
    return truthValue(2, option)

def inhabitantTruths(person, statements = options):
    truths = []
    for o in statements:
        if o[0] == o[person]:
            truths.append(o)
    return truths

def inhabitantLies(person, statement = options):
    lies = []
    for o in statement:
        if o[0] != o[person]:
            lies.append(o)
    return lies

def complement(s):
    comp = []
    for o in options:
        if (o[0]!=s[0] or o[1]!= s[1] or o[2]!=s[2]):
            comp.append(o)
    return comp

def reducedAdd(listOne, listTwo):
    reduced = [] + listTwo
    for l in listOne:
        if l not in listTwo:
            reduced.append(l)
    return reduced

def fullComplement(statements):
    comp = []
    for s in statements:
        comp = reducedAdd(comp, complement(s))
    for s in statements:
        if s in comp:
            comp.remove(s)
    return comp

def intersection(set1,set2):
    intersect = []
    for s in set1:
        if s in set2:
            intersect.append(s)
    return intersect



Next we define functions for generating the structures that go with different types of statements.

In [40]:
def dayAlone(day):
    statements = []
    for i in time:
        for j in time:
            statements.append([day, i, j])
    return statements
 
def dayAloneStatement(day):
    return "It is " + day + "."

def selfAlone(person, self):
    a = ['blank','blank','blank']
    a[0]= 'day'
    a[person] = self
    a[2 -math.floor(person/2) % 2] = 'day';
    b = ['blank','blank','blank']
    b[0] = 'day'
    b[person] = self
    b[2 -math.floor(person/2) % 2] = 'night';
    c = ['blank','blank','blank']
    c[0]= 'night'
    c[person] = self
    c[2 -math.floor(person/2) % 2] = 'day';
    d = ['blank','blank','blank']
    d[0]= 'night'
    d[person] = self
    d[2 -math.floor(person/2) % 2] = 'night';   
    return [a,b,c,d]
    
def selfAloneStatement(day):
    return "I am a " + day + "-knight."

#the person is making a statement about themselves and the day
def selfAndDay(person, self, day):
    a = ['blank','blank','blank']
    a[0]= day
    a[person] = self
    a[2 -math.floor(person/2) % 2] = 'day';
    b = ['blank','blank','blank']
    b[0] = day
    b[person] = self
    b[2 -math.floor(person/2) % 2] = 'night';
    return [a,b]

def selfAndDayStatement(self, day):
    return "I am a " + self + "-knight, and it is " + day + "."


# the person is making a statement about the other and the day 
def otherAndDay(person, other, day):
    a = ['blank','blank','blank']
    a[0]= day
    a[person] = 'day'
    a[2 -math.floor(person/2) % 2] = other;
    b = ['blank','blank','blank']
    b[0] = day
    b[person] = 'night'
    b[2 -math.floor(person/2) % 2] = other;
    return [a,b]

def otherAndDayStatement(other, day):
    return "The other persion is a " + other + "-knight, and it is " + day + "."

# the person is making a statement themselves and the other 
def otherAndSelf(person, self, other):
    a = ['blank','blank','blank']
    a[0]= 'day'
    a[person] = self
    a[2 -math.floor(person/2) % 2] = other;
    b = ['blank','blank','blank']
    b[0] = 'night'
    b[person] = self
    b[2 -math.floor(person/2) % 2] = other;
    return [a,b]

def otherAndSelfStatement(other, self):
    return "I am a " + self + "-knight, and the other persion is a " + other + "-knight."

Next, create two lists of statements, one for the first person and one for the second.

In [44]:
person1Statements = []
person2Statements = []
for p in time:
    for d in time:        
        person1Statements.append({'statement': selfAndDayStatement(p,d), 'state': selfAndDay(1,p,d)})
        person1Statements.append({'statement': otherAndDayStatement(p,d), 'state': otherAndDay(1,p,d)})
        person1Statements.append({'statement': otherAndSelfStatement(p,d), 'state': otherAndSelf(1,p,d)})
        person2Statements.append({'statement': selfAndDayStatement(p,d), 'state': selfAndDay(2,p,d)})
        person2Statements.append({'statement': otherAndDayStatement(p,d), 'state': otherAndDay(2,p,d)})
        person2Statements.append({'statement': otherAndSelfStatement(p,d), 'state': otherAndSelf(2,p,d)})
        person1Statements.append({'statement': dayAloneStatement(d), 'state': dayAlone(d)})
        person2Statements.append({'statement': dayAloneStatement(d), 'state': dayAlone(d)})
        person1Statements.append({'statement': selfAloneStatement(d), 'state': selfAlone(1,p)})
        person2Statements.append({'statement': selfAloneStatement(d), 'state': selfAlone(2,p)})
print("Each person has " + str(len(person1Statements)) + " statements they can make.")

Each person has 20 statements they can make.


Finally, we check all combinations of statements from person 1 and person 2, and see which ones generate a unique solution - these will be the puzzles.

In [45]:
solutionCount = 0
for s1 in person1Statements:
    for s2 in person2Statements:
        ts1 = inhabitantTruths(1, s1['state'])
        fs1 = inhabitantLies(1, fullComplement(s1['state']))
        allS1 = reducedAdd(ts1,fs1)
        ts2 = inhabitantTruths(2, s2['state'])
        fs2 = inhabitantLies(2, fullComplement(s2['state']))
        allS2 = reducedAdd(ts2,fs2)
        solution = intersection(allS1, allS2)
        if len(solution) ==1 :
            solutionCount = solutionCount +1
            print("puzzle found")
            print("inhabitant 1 says: " + s1['statement'])
            print("if she is telling the truth: " + str(ts1))
            print("if she is lying: " + str(fs1))
            
            print("inhabitant 2 says: " + s2['statement'])
            print("if she is telling the truth: " + str(ts2))
            print("if she is lying: " + str(fs2))
            
            print('solution: ' + str(solution[0]))
            print('---------------')
print('We generated ' + str(solutionCount) + " puzzles.")

puzzle found
inhabitant 1 says: I am a day-knight, and it is day.
if she is telling the truth: [['day', 'day', 'day'], ['day', 'day', 'night']]
if she is lying: [['day', 'night', 'day'], ['day', 'night', 'night'], ['night', 'day', 'day'], ['night', 'day', 'night']]
inhabitant 2 says: I am a night-knight, and it is day.
if she is telling the truth: []
if she is lying: [['night', 'day', 'day'], ['night', 'night', 'day']]
solution: ['night', 'day', 'day']
---------------
puzzle found
inhabitant 1 says: The other persion is a day-knight, and it is day.
if she is telling the truth: [['day', 'day', 'day']]
if she is lying: [['day', 'night', 'night'], ['night', 'day', 'day'], ['night', 'day', 'night']]
inhabitant 2 says: I am a day-knight, and it is night.
if she is telling the truth: []
if she is lying: [['day', 'day', 'night'], ['day', 'night', 'night']]
solution: ['day', 'night', 'night']
---------------
puzzle found
inhabitant 1 says: The other persion is a day-knight, and it is day.
if s